<div style="font-family: 'Arial', sans-serif; margin: 0px; padding: 30px; background-color: #FFF3CD; border-left: 5px solid #FFECB3; border-radius: 50px;">
    <h2 style="color: #856404; font-size: 24px; margin-top: 0;">🐔 Chickens and Eggs: A System Dynamics Model 🥚</h2>
    <p style="color: #6c757d; font-size: 16px;">
        Welcome to the interactive <strong>Chickens and Eggs</strong> model! This tool was designed by Robinson Salazar Rua and Peter Hovmand to help you understand the fascinating dynamics of a chicken population.
    </p>
    <p style="color: #6c757d; font-size: 16px;">
        In this model, you'll explore how different factors (such as fertility rate, death risk, and capacity level) affect the population of chickens in our virtual farm.
    </p>
    <h3 style="color: #856404; font-size: 18px;"> Your Tasks:</h3>
    <ul style="color: #6c757d; font-size: 16px;">
        <li><strong>Adjust the Parameters:</strong> Use the sliders to change the model initial conditions.</li>
        <li><strong>Observe the Changes:</strong> Watch how these adjustments impact the chicken population over time.</li>
        <li><strong>Discover the Dynamics:</strong> Notice the feedback loops and delays that characterize this system.</li>
    </ul>
    <p style="color: #6c757d; font-size: 16px;">
        By playing with these parameters, you'll gain insights into the principles of system dynamics and understand how even simple systems can exhibit complex behaviors. Enjoy your journey into the world of chickens and eggs!
    </p>
</div>


In [ ]:
%%capture
!pip install pysd
!pip install netCDF4
!pip install voila --user

In [ ]:
import os
import pysd
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import display, HTML

In [ ]:
def run_model(normal_fertility_rate, normal_death_risk, max_chicken_capacity, initial_chicken_population, initial_eggs_population):
    
    model= pysd.read_vensim("SFD chicken model V2.mdl")
    params = {'Normal fertility rate': normal_fertility_rate, 'Normal death risk': normal_death_risk, 'Max chicken capacity': max_chicken_capacity, 'Initial chicken population':initial_chicken_population, 'Initial number of eggs':initial_eggs_population}
    return_columns = ['Chickens', 'Eggs', 'Births', 'Deaths']
    return_timestamps = range(51)

    results = model.run(params=params, return_columns=return_columns, return_timestamps=return_timestamps)
    
    display(HTML("<h2 style='margin: 0 0 15px 0; font-family: Comic Sans MS;'>Simulation Results</h2>"))
    
    results['Chickens'].plot(figsize=(10, 5))
    results['Eggs'].plot(figsize=(10, 5))
    plt.xlabel('Time (Days)')
    plt.ylabel('Number of Chickens and Eggs')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()
    
    results['Births'].plot(figsize=(10, 5), color='green', label='Births')
    results['Deaths'].plot(figsize=(10, 5), color='red', label='Deaths')
    plt.xlabel('Time (Days)')
    plt.ylabel('Number of Chickens per Day')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()   
    
    concatenated_results = pd.concat([results.head(6), results.tail(6)])
    concatenated_html = concatenated_results.to_html(classes='table table-striped')
    
    styled_html = f"""
    <div style='margin: auto; width: 100%; padding: 5px;'>
        {concatenated_html}
    </div>
    """
    display(HTML(styled_html))

In [ ]:
# Create a title widget
title = HTML("<h2 style='margin: 0 0 15px 0; font-family: Comic Sans MS;'>Parameters</h2>")

# Common layout settings for all sliders
slider_layout = widgets.Layout(width='300px', height='100px', margin='0 0 5px 0')

# Slider style
slider_style = {'handle_color': '#856404', 'description_width': 'initial'}

# Define a custom style for the slider descriptions
description_style = {'description_font': 'Comic Sans MS'}

# Create the sliders
fertility_rate_slider = widgets.FloatSlider(
    value=0.2, min=0, max=1, step=0.01,
    description='Normal Fertility Rate', style=slider_style, layout=slider_layout, description_style=description_style
)

death_risk_slider = widgets.FloatSlider(
    value=0.2, min=0, max=1.0, step=0.01,
    description='Normal Death Risk', style=slider_style, layout=slider_layout, description_style=description_style
)

max_chicken_capacity_slider = widgets.FloatSlider(
    value=1000, min=0, max=2000, step=10,
    description='Max Chicken Capacity', style=slider_style, layout=slider_layout, description_style=description_style
)

initial_chicken_population_slider = widgets.FloatSlider(
    value=1000, min=0, max=4000, step=10,
    description='Initial Chicken Population', style=slider_style, layout=slider_layout, description_style=description_style
)

initial_eggs_population_slider = widgets.FloatSlider(
    value=1000, min=0, max=2000, step=10,
    description='Initial Number of Eggs', style=slider_style, layout=slider_layout, description_style=description_style
)

# Define the reset function
def reset_sliders(b):
    fertility_rate_slider.value = 0.2
    death_risk_slider.value = 0.2
    max_chicken_capacity_slider.value = 1000
    initial_chicken_population_slider.value = 1000
    initial_eggs_population_slider.value = 1000

# Create the reset button
reset_button = widgets.Button(
    description='Reset All',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click here to reset all the inputs and outputs to their initial values',
    icon='refresh', # FontAwesome icon name
    layout=widgets.Layout(width='200px', height='40px', margin='10px')  # Adjust the width, height, and margin as needed
)

# Link the button to the reset function
reset_button.on_click(reset_sliders)

# Organize the sliders into two columns
left_column = widgets.VBox([fertility_rate_slider, death_risk_slider, max_chicken_capacity_slider], layout=slider_layout)
right_column = widgets.VBox([initial_chicken_population_slider, initial_eggs_population_slider], layout=slider_layout)

controls = HBox([left_column, right_column, reset_button])

# Link sliders to the model function
interactive_output_widget = widgets.interactive_output(run_model, {
    'normal_fertility_rate': fertility_rate_slider,
    'normal_death_risk': death_risk_slider,
    'max_chicken_capacity': max_chicken_capacity_slider,
    'initial_chicken_population': initial_chicken_population_slider,
    'initial_eggs_population': initial_eggs_population_slider
})

# Display the columns side by side
display(title, controls, interactive_output_widget)